<a href="https://colab.research.google.com/github/Smusyc/Synthetic-Media/blob/main/Lab_6_Sotin_synthetic_media.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install clearml
!pip install np_utils

In [ ]:
!pip install np_utils

In [ ]:
import numpy as np
import re
from IPython.display import clear_output
import os
import keras
from keras.layers import Dense, LSTM, Input, Embedding, Dropout
from keras.utils import to_categorical
from keras.models import Model, load_model
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import LambdaCallback

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import clearml
from clearml import Task, Logger
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, ParameterSampler
from sklearn.metrics import roc_auc_score

Mounted at /content/drive


In [ ]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=PII6865ZQUVOMWNU3NYY
%env CLEARML_API_SECRET_KEY=wZXjcAeQGT8AdD92Reuh7bDsOppveMXWaoWOJjknQFqOYkW1fo

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=PII6865ZQUVOMWNU3NYY
env: CLEARML_API_SECRET_KEY=wZXjcAeQGT8AdD92Reuh7bDsOppveMXWaoWOJjknQFqOYkW1fo


In [ ]:
task = Task.init(
    project_name='Esop_Gen',
    task_name='TextGeneration',
    tags=['TextGeneration','keras','LSTM'])

ClearML Task: created new task id=08ccf2dec4ec4f98b31f9004d507acec
2023-11-30 13:30:01,104 - clearml.Task - INFO - No repository found, storing script code instead
ClearML results page: https://app.clear.ml/projects/229327f4af1d4c56b4ddda8b0274e57b/experiments/08ccf2dec4ec4f98b31f9004d507acec/output/log


In [ ]:
import keras

In [ ]:
load_saved_model = True
train_model = False

In [ ]:
token_type = 'word'

In [ ]:
sh = """
FILE=$1
NAME=$2
echo FILE
echo NAME
URL=http://www.gutenberg.org/cache/epub/$FILE/pg$FILE.txt
TARGET_DIR=./data/$NAME/
mkdir $TARGET_DIR
TXT_FILE=./data/$NAME/data.txt
wget -N $URL -O $TXT_FILE
"""
with open('script.sh', 'w') as file:
  file.write(sh)

!bash script.sh 21 esop_txt
!cp ./data/esop_txt/data.txt /content/drive/MyDrive/SyntheticMedia/Dataset/data.txt

FILE
NAME
mkdir: cannot create directory ‘./data/esop_txt/’: No such file or directory
for details.

./data/esop_txt/data.txt: No such file or directory
cp: cannot stat './data/esop_txt/data.txt': No such file or directory


In [ ]:
#load in the text and perform some cleanup
seq_length = 20

filename = "./drive/MyDrive/SyntheticMedia/Dataset/data.txt"

with open(filename, encoding='utf-8-sig') as f:
    text = f.read()

In [ ]:
text = text.lower()

In [ ]:
start = text.find("the wolf and the lamb")
end = text.find("footnotes")
text = text[start:end]

In [ ]:
start_story = '| ' * seq_length

text = start_story + text
text = text.lower()
text = text.replace('\n\n\n\n\n', start_story)
text = text.replace('\n', ' ')
text = text.replace('"', ' ')
text = text.replace(' ,', '')
text = text.replace('“', '').replace('”', '')
text = re.sub('  +', '. ', text).strip()
text = text.replace('..', '.')

text = re.sub('([!"“”#$%&()*+,,-./:;<=>?@[\]^_`{|}~])', r' \1 ', text)
text = re.sub('\s{2,}', ' ', text)

In [ ]:
len(text)

232111

In [ ]:
text

' | | | | | | | | | | | | | | | | | | | | the wolf and the lamb . wolf , meeting with a lamb astray from the fold , resolved not to lay violent hands on him , but to find some plea to justify to the lamb the wolf’s right to eat him . he thus addressed him : sirrah , last year you grossly insulted me . indeed , bleated the lamb in a mournful tone of voice , i was not then born . then said the wolf , you feed in my pasture . no , good sir , replied the lamb , i have not yet tasted grass . again said the wolf , you drink of my well . no , exclaimed the lamb , i never yet drank water , for as yet my mother’s milk is both food and drink to me . upon which the wolf seized him and ate him up , saying , well ! i won’t remain supperless , even though you refute every one of my imputations . the tyrant will always find a pretext for his tyranny . | | | | | | | | | | | | | | | | | | | | the bat and the weasels . a bat who fell upon the ground and was caught by a weasel pleaded to be spared his li

In [ ]:

if token_type == 'word':
    tokenizer = Tokenizer(char_level = False, filters = '')
else:
    tokenizer = Tokenizer(char_level = True, filters = '', lower = False)


tokenizer.fit_on_texts([text])

total_words = len(tokenizer.word_index) + 1

token_list = tokenizer.texts_to_sequences([text])[0]

total_words

print(tokenizer.word_index)
print(token_list)

import keras


{'|': 1, 'the': 2, ',': 3, '.': 4, 'and': 5, 'a': 6, 'to': 7, 'of': 8, 'his': 9, 'in': 10, 'he': 11, 'you': 12, 'him': 13, 'that': 14, 'with': 15, 'i': 16, 'for': 17, 'it': 18, 'as': 19, 'was': 20, 'by': 21, 'not': 22, 'on': 23, 'had': 24, 'said': 25, 'but': 26, 'is': 27, 'at': 28, 'be': 29, 'they': 30, 'from': 31, 'one': 32, 'when': 33, 'them': 34, 'an': 35, 'her': 36, 'have': 37, 'their': 38, 'who': 39, 'which': 40, 'me': 41, 'all': 42, 'my': 43, ';': 44, '-': 45, 'lion': 46, 'this': 47, ':': 48, 'your': 49, 'if': 50, 'up': 51, 'so': 52, 'into': 53, 'man': 54, 'fox': 55, 'would': 56, 'will': 57, 'were': 58, 'do': 59, 'are': 60, 'ass': 61, '?': 62, 'himself': 63, 'out': 64, 'no': 65, 'wolf': 66, 'upon': 67, 'what': 68, 'she': 69, 'should': 70, 'replied': 71, 'these': 72, 'made': 73, 'time': 74, 'its': 75, 'some': 76, 'only': 77, 'very': 78, '!': 79, 'day': 80, 'own': 81, 'could': 82, 'fables': 83, 'being': 84, 'down': 85, 'great': 86, 'thus': 87, 'saw': 88, 'am': 89, 'while': 90, 'cam

In [ ]:
total_words

5221

In [ ]:
def generate_sequences(token_list, step):

    X = []
    y = []

    for i in range(0, len(token_list) - seq_length, step):
        X.append(token_list[i: i + seq_length])
        y.append(token_list[i + seq_length])


    y = keras.utils.to_categorical(y, num_classes = total_words)

    num_seq = len(X)
    print('Number of sequences:', num_seq, "\n")

    return X, y, num_seq

step = 1
seq_length = 20

X, y, num_seq = generate_sequences(token_list, step)

X = np.array(X)
y = np.array(y)


Number of sequences: 52257 



In [ ]:
X.shape

(52257, 20)

In [ ]:
y.shape

(52257, 5221)

## Define the LSTM model

In [ ]:
load_saved_model=False
if load_saved_model:
    # model = load_model('./saved_models/lstm_esop_gen_dropout_100_1.h5')
    model = load_model('/drive/MyDrive/SyntheticMedia/Models/my_model_weights_d15.h5')

else:

    n_units = 256
    embedding_size = 100

    text_in = Input(shape = (None,))
    embedding = Embedding(total_words, embedding_size)
    x = embedding(text_in)
    x = LSTM(n_units)(x)
    # x = Dropout(0.2)(x)
    text_out = Dense(total_words, activation = 'softmax')(x)

    model = Model(text_in, text_out)

    opti = RMSprop(lr = 0.001)
    model.compile(loss='categorical_crossentropy', optimizer=opti)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         522100    
                                                                 
 lstm (LSTM)                 (None, 256)               365568    
                                                                 
 dense (Dense)               (None, 5221)              1341797   
                                                                 
Total params: 2229465 (8.50 MB)
Trainable params: 2229465 (8.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
def sample_with_temp(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)



def generate_text(seed_text, next_words, model, max_sequence_len, temp):
    output_text = seed_text

    seed_text = start_story + seed_text

    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = token_list[-max_sequence_len:]
        token_list = np.reshape(token_list, (1, max_sequence_len))

        probs = model.predict(token_list, verbose=0)[0]
        y_class = sample_with_temp(probs, temperature = temp)

        if y_class == 0:
            output_word = ''
        else:
            output_word = tokenizer.index_word[y_class]

        if output_word == "|":
            break

        if token_type == 'word':
            output_text += output_word + ' '
            seed_text += output_word + ' '
        else:
            output_text += output_word + ' '
            seed_text += output_word + ' '


    return output_text

In [ ]:
train_model = True

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         522100    
                                                                 
 lstm (LSTM)                 (None, 256)               365568    
                                                                 
 dense (Dense)               (None, 5221)              1341797   
                                                                 
Total params: 2229465 (8.50 MB)
Trainable params: 2229465 (8.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#history=0
def on_epoch_end(epoch, logs):
    seed_text = ""
    gen_words = 500

    print('Temp 0.2')
    print (generate_text(seed_text, gen_words, model, seq_length, temp = 0.2))
    print('Temp 0.33')
    print (generate_text(seed_text, gen_words, model, seq_length, temp = 0.33))
    print('Temp 0.5')
    print (generate_text(seed_text, gen_words, model, seq_length, temp = 0.5))
    print('Temp 1.0')
    print (generate_text(seed_text, gen_words, model, seq_length, temp = 1))



if train_model:
    epochs = 13
    batch_size = 32
    num_batches = int(len(X) / batch_size)
    callback = LambdaCallback(on_epoch_end=on_epoch_end)
    history = model.fit(X, y, epochs=epochs, validation_split=0.1, batch_size=batch_size, callbacks = [callback], shuffle = True,  verbose=2)




Epoch 1/13
Temp 0.2
the and and and the man . a fox , a ass , a fox , a fox , the man , a lion , the fox , the fox , the fox , the fox , the fox , the said , the fox , the lion , the fox , the man , the fox , and , a fox , the lion , the lion , the fox , the fox , the lion , the fox , the fox , the fox , the fox , the fox , the lion , the fox , the fox , the lion , the fox , the lion , the fox , the fox , the man , the fox , the fox , the fox , a man , the fox , the fox , the of , and the fox , the lion , the man , the fox , the fox , a lion , the fox , the lion , the fox . the man , the man , the fox , the lion , a man , the fox , the fox , the fox , the fox , the fox , the fox , the fox , the lion , the fox , the fox , the ass , the fox , the fox , the fox , the fox , the fox , the man , the fox , the fox , the fox , the fox , the own , and the man , the fox , and the fox , the fox , the of , and the lion , the fox , the fox , the fox , the fox , the man , the fox , the fox , the ass

AttributeError: ignored

In [ ]:
#model.history.keys()

print(history.history.keys())

dict_keys(['loss', 'val_loss'])


In [ ]:
model.history.history.keys()

dict_keys(['loss', 'val_loss'])

In [ ]:
logger = task.get_logger()

In [ ]:
import os
import keras

In [ ]:
for i in range(len(model.history.history['loss'])):
    logger.report_scalar("loss_d15", "train", iteration=i, value=model.history.history['loss'][i])
    logger.report_scalar("loss_d15", "test", iteration=i, value=model.history.history['val_loss'][i])
    logger.report_scalar("auc_d15", "train", iteration=i, value=model.history.history['auc'][i])
    logger.report_scalar("auc_d15", "test", iteration=i, value=model.history.history['val_auc'][i])
    logger.report_scalar("precision_d15", "train", iteration=i, value=model.history.history['precision'][i])
    logger.report_scalar("precision_d15", "test", iteration=i, value=model.history.history['val_precision'][i])
    logger.report_scalar("recall_d15", "train", iteration=i, value=model.history.history['recall'][i])
    logger.report_scalar("recall_d15", "test", iteration=i, value=model.history.history['val_recall'][i])
logger.flush()

True

In [ ]:
model.save_weights('/content/drive/MyDrive/SyntheticMedia/Models/my_model_weights_d15.h5')

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         522100    
                                                                 
 lstm (LSTM)                 (None, 256)               365568    
                                                                 
 dense (Dense)               (None, 5221)              1341797   
                                                                 
Total params: 2229465 (8.50 MB)
Trainable params: 2229465 (8.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
def inferens(input_text, len):
  temp = 0.1
  print (generate_text(input_text, len, model, seq_length, temp))



In [ ]:
inferens('One day the cat and ', 100)

One day the cat and the fox . a fox , a lion , and a lion , and not to the shepherd . the fox , and the eagle , and the shepherd , he said to a lion , you would be a little - tree , and you will be the man of the ass . the ass , and the man , and a man , and not to the shepherd of the his , and the fox , and the man was the tree . the ass , the fox , and the fox , and the shepherd , he 


In [ ]:
#seed_text = "the frog and the snake . "
#gen_words = 500
#temp = 0.1
#
#print (generate_text(seed_text, gen_words, model, seq_length, temp))#

In [ ]:
#def generate_human_led_text(model, max_sequence_len):
#
#    output_text = ''
#    seed_text = start_story
#
#    while 1:
#        token_list = tokenizer.texts_to_sequences([seed_text])[0]
#        token_list = token_list[-max_sequence_len:]
#        token_list = np.reshape(token_list, (1, max_sequence_len))
#
#        probs = model.predict(token_list, verbose=0)[0]
#
#        top_10_idx = np.flip(np.argsort(probs)[-10:])
#        top_10_probs = [probs[x] for x in top_10_idx]
#        top_10_words = tokenizer.sequences_to_texts([[x] for x in top_10_idx])
#
#        for prob, word in zip(top_10_probs, top_10_words):
#            print('{:<6.1%} : {}'.format(prob, word))
#
#        chosen_word = input()
#
#        if chosen_word == '|':
#            break
#
#
#        seed_text += chosen_word + ' '
#        output_text += chosen_word + ' '

        #clear_output()

        #print (output_text)




In [ ]:
#generate_human_led_text(model, 20)